### 흐름을 생각해보자

BM25로 문장을 embedding한 후에 각 vector들에 대한 self.embedding과 keyword의 embedding 
행렬곱을 normalization 한 후에 가장 높은 것 + 일정 수치 이상

1. normalization 값을 어떻게 할 것인가
 - 각 키워드에 대한 행렬곱을 살펴보고 어느 정도 수치가 나오는지 정성적 평가 -> threshold 정하기
 - 문장이 "모르겠어요." 혹은 얼토당토하지 않은 문장을 적었음에도 top 순위로 하면 키워드를 필수적으로 찾아야하므로 오히려 오답을 매칭하는 경우가 있을 것
 - 그렇다고 threshold를 너무 높이다보면 오히려 EM으로 찾을 수 있는 단어들만이 출현할 것
 - 결국 정성적인 평가밖에 방식이 없나 싶다.

2. 일정 수치는 어떻게 정할 것인가

Dense Retriever의 경우 우리의 키워드가 문장의 어떤 단어와 연관이 높은지를 학습시켜줄 label이 필요하다
ex) 기온 - 온도가 연관이 있다는 label 필요
하지만 현재 그러한 데이터셋이 존재하지 않는 상황이기 때문에 이를 위한 작업은 너무 많은 시간을 소요하게 될 것 같다.
따라서 이는 제외하는 것이 맞아보인다.

결과적으로 BM25로만의 embedding 이후에 결과를 살펴봐야 할 것 같다

In [1]:
import numpy as np
import pandas as pd
import torch
import os
import pickle
import time
from typing import List, NoReturn, Optional, Tuple, Union
import rank_bm25
from datasets import Dataset, concatenate_datasets, load_from_disk
import time
import torch.nn.functional as F
from torch.nn.functional import softmax
from tqdm.auto import tqdm
from transformers import AutoTokenizer, BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup
from sklearn.feature_extraction.text import TfidfVectorizer


class MyBm25(rank_bm25.BM25Okapi): 
                                                                      
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        # 논문에 따르면 k1은 1.2~1.5가 적당하다. 또한 b의 경우에도 0.75~0.9가 적당하다.
        # GridSearchCV를 통해서 최적의 값을 찾아보고 정의하는 것이 좋을 듯 하다.
            super().__init__(corpus, tokenizer=tokenizer, k1=k1, b=b, epsilon=epsilon)    
    
    def get_top_n(self, query, documents, n=5):
        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        # 이미 구현되어 있는 함수를 사용하여 점수를 구한다.

        top_n_idx = np.argsort(scores)[::-1][:n]
        doc_score = scores[top_n_idx]
        
        return doc_score, top_n_idx

In [39]:

df = pd.read_csv("../data/" + "KorSTS/sts-train.tsv", sep='\t')
contexts1 = list(
            df.iloc[i].sentence1 for i in range(len(df))
        )  # set 은 매번 순서가 바뀌므로
contexts2 = list(df.iloc[j].sentence2 for j in range(len(df)))
contexts = contexts1 + contexts2
for idx, i in enumerate(contexts):
    if type(i) == float:
        contexts.pop(idx)

8216
8344
8645
8772
8825
9067


In [46]:
class SparseRetrieval:
    def __init__(
        self,
        tokenize_fn,
        data_path: Optional[str] = "../data/",
        context_path: Optional[str] = "KorSTS/sts-train.tsv",
        k1=1.5, b=0.75, epsilon=0.25,
        is_bm25 = True
    ) -> NoReturn:

        """
        Arguments:
            tokenize_fn:
                기본 text를 tokenize해주는 함수입니다.
                아래와 같은 함수들을 사용할 수 있습니다.
                - lambda x: x.split(' ')
                - Huggingface Tokenizer
                - konlpy.tag의 Mecab
            data_path:
                데이터가 보관되어 있는 경로입니다.
            context_path:
                Passage들이 묶여있는 파일명입니다.
            data_path/context_path가 존재해야합니다.
        Summary:
            Passage 파일을 불러오고 TfidfVectorizer를 선언하는 기능을 합니다.
        """

        self.data_path = data_path
        df = pd.read_csv(data_path + context_path, sep='\t')

        self.contexts1 = list(
            df.iloc[i].sentence1 for i in range(len(df))
        )  # set 은 매번 순서가 바뀌므로
        self.contexts2 = list(df.iloc[j].sentence2 for j in range(len(df)))
        self.contexts = self.contexts1 + self.contexts2
        
        for idx, i in enumerate(self.contexts):
            if type(i) == float:
                self.contexts.pop(idx)
        

        print(f"Lengths of unique contexts : {len(self.contexts)}")
        self.ids = list(range(len(self.contexts)))

        # Transform by vectorizer
        self.tfidfv = TfidfVectorizer(
            tokenizer=tokenize_fn, ngram_range=(1, 2), max_features=50000,
        )
        self.tokenize_fn = tokenize_fn

        self.p_embedding = None  # get_sparse_embedding()로 생성합니다
        self.indexer = None  # build_faiss()로 생성합니다.
        self.bm25 = None
        self.is_bm25 = is_bm25
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        self.get_sparse_embedding() 

    def get_sparse_embedding(self) -> NoReturn:

        """
        Summary:
            Passage Embedding을 만들고
            TFIDF와 Embedding을 pickle로 저장합니다.
            만약 미리 저장된 파일이 있으면 저장된 pickle을 불러옵니다.
        """
        if not self.is_bm25: # tfidf를 사용하는 경우
        # Pickle을 저장합니다.
            pickle_name = f"sparse_embedding.bin"
            tfidfv_name = f"tfidv.bin"
            emd_path = os.path.join(self.data_path, pickle_name)
            tfidfv_path = os.path.join(self.data_path, tfidfv_name)

            if os.path.isfile(emd_path) and os.path.isfile(tfidfv_path):
                with open(emd_path, "rb") as file:
                    self.p_embedding = pickle.load(file)
                with open(tfidfv_path, "rb") as file:
                    self.tfidfv = pickle.load(file)
                print("Embedding pickle load.")
            else:
                print("Build passage embedding")
                self.p_embedding = self.tfidfv.fit_transform(self.contexts)
                print(self.p_embedding.shape)
                with open(emd_path, "wb") as file:
                    pickle.dump(self.p_embedding, file)
                with open(tfidfv_path, "wb") as file:
                    pickle.dump(self.tfidfv, file)
                print("Embedding pickle saved.")
        
        else: # bm25
            bm25_name = f"bm25.bin"
            bm25_path = os.path.join(self.data_path, bm25_name)
            if os.path.isfile(bm25_path):
                with open(bm25_path, "rb") as file:
                    self.bm25 = pickle.load(file)
                print("Embedding bm25 pickle load.")
            else:
                print("Building bm25... It may take 1 minute and 30 seconds...")
                # bm25 must tokenizer first 
                # because it runs pool inside and this cuases unexpected result.
                tokenized_corpus = []
                for c in tqdm(self.contexts):
                    tokenized_corpus.append(self.tokenize_fn(c))
                self.bm25 = MyBm25(tokenized_corpus, k1 = self.k1, b = self.b, epsilon=self.epsilon)
                # bm25 클래스를 불러와서 실행합니다.
                
                with open(bm25_path, "wb") as file:
                    pickle.dump(self.bm25, file)
                print("bm25 pickle saved.")
    
    def retrieve(
        self, query_or_dataset: Union[str, Dataset], topk: Optional[int] = 1
    ) -> Union[Tuple[List, List], pd.DataFrame]:

        """
        Arguments:
            query_or_dataset (Union[str, Dataset]):
                str이나 Dataset으로 이루어진 Query를 받습니다.
                str 형태인 하나의 query만 받으면 `get_relevant_doc`을 통해 유사도를 구합니다.
                Dataset 형태는 query를 포함한 HF.Dataset을 받습니다.
                이 경우 `get_relevant_doc_bulk`를 통해 유사도를 구합니다.
            topk (Optional[int], optional): Defaults to 1.
                상위 몇 개의 passage를 사용할 것인지 지정합니다.
        Returns:
            1개의 Query를 받는 경우  -> Tuple(List, List)
            다수의 Query를 받는 경우 -> pd.DataFrame: [description]
        Note:
            다수의 Query를 받는 경우,
                Ground Truth가 있는 Query (train/valid) -> 기존 Ground Truth Passage를 같이 반환합니다.
                Ground Truth가 없는 Query (test) -> Retrieval한 Passage만 반환합니다.
        """

        assert self.p_embedding is not None, "get_sparse_embedding() 메소드를 먼저 수행해줘야합니다."

        if isinstance(query_or_dataset, str):
            doc_scores, doc_indices = self.get_relevant_doc(query_or_dataset, k=topk)
            print("[Search query]\n", query_or_dataset, "\n")

            for i in range(topk):
                print(f"Top-{i+1} passage with score {doc_scores[i]:4f}")
                print(self.contexts[doc_indices[i]])

            return (doc_scores, [self.contexts[doc_indices[i]] for i in range(topk)])

        elif isinstance(query_or_dataset, Dataset):

            # Retrieve한 Passage를 pd.DataFrame으로 반환합니다.
            total = []
            with timer("query exhaustive search"):
                doc_scores, doc_indices = self.get_relevant_doc_bulk(
                    query_or_dataset["question"], k=topk
                )
            for idx, example in enumerate(
                tqdm(query_or_dataset, desc="Sparse retrieval: ")
            ):
                tmp = {
                    # Query와 해당 id를 반환합니다.
                    "question": example["question"],
                    "id": example["id"],
                    # Retrieve한 Passage의 id, context를 반환합니다.
                    "context_id": doc_indices[idx],
                    "context": " ".join(
                        [self.contexts[pid] for pid in doc_indices[idx]]
                    ),
                }
                if "context" in example.keys() and "answers" in example.keys():
                    # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
                    tmp["original_context"] = example["context"]
                    tmp["answers"] = example["answers"]
                total.append(tmp)

            cqas = pd.DataFrame(total)
            return cqas
    
    def get_relevant_doc(self, query: str, k: Optional[int] = 1) -> Tuple[List, List]:

        """
        Arguments:
            query (str):
                하나의 Query를 받습니다.
            k (Optional[int]): 1
                상위 몇 개의 Passage를 반환할지 정합니다.
        Note:
            vocab 에 없는 이상한 단어로 query 하는 경우 assertion 발생 (예) 뙣뙇?
        """

        with timer("transform"):
            query_vec = self.tfidfv.transform([query])
        assert (
            np.sum(query_vec) != 0
        ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

        with timer("query ex search"):
            result = query_vec * self.p_embedding.T
        if not isinstance(result, np.ndarray):
            result = result.toarray()

        sorted_result = np.argsort(result.squeeze())[::-1]
        doc_score = result.squeeze()[sorted_result].tolist()[:k]
        doc_indices = sorted_result.tolist()[:k]
        return doc_score, 


In [47]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large", use_fast=False,)

retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    )

Lengths of unique contexts : 11472
Building bm25... It may take 1 minute and 30 seconds...



bm25 pickle saved.


In [ ]:
org_dataset = load_from_disk(dataset_name)
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)

In [ ]:
with timer("bulk query by exhaustive search"):
    df = retriever.retrieve(full_ds)
    df["correct"] = df["original_context"] == df["context"]
    print(
        "correct retrieval result by exhaustive search",
        df["correct"].sum() / len(df),
        )

with timer("single query by exhaustive search"):
            scores, indices = retriever.retrieve(query)

새롭게 BM25를 만들어보자

In [ ]:
import math

class BM25:
    """
    Best Match 25.

    Parameters
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    Attributes
    ----------
    tf_ : list[dict[str, int]]
        Term Frequency per document. So [{'hi': 1}] means
        the first document contains the term 'hi' 1 time.

    df_ : dict[str, int]
        Document Frequency per term. i.e. Number of documents in the
        corpus that contains the term.

    idf_ : dict[str, float]
        Inverse Document Frequency per term.

    doc_len_ : list[int]
        Number of terms per document. So [3] means the first
        document contains 3 terms.

    corpus_ : list[list[str]]
        The input corpus.

    corpus_size_ : int
        Number of documents in the corpus.

    avg_doc_len_ : float
        Average number of terms for documents in the corpus.
    """

    def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

    def fit(self, corpus):
        """
        Fit the various statistics that are required to calculate BM25 ranking
        score using the corpus given.

        Parameters
        ----------
        corpus : list[list[str]]
            Each element in the list represents a document, and each document
            is a list of the terms.

        Returns
        -------
        self
        """
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

    def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

    def keyword_search(self, query, index, threshold=0.5):
        scores = [self.keyword_score(query, index)]
        return scores

    def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score
    def keyword_score(self, query, index):
        
        score = []

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score.append((numerator / denominator))

        return score

### Word2Vec을 통한 임베딩 진행 

https://doitgrow.com/27

python3 -m pip install konlpy

pip install beautifulsoup4

pip install --upgrade gensim

pip install soynlp

apt-get update

apt install default-jdk

pip install git+https://github.com/haven-jeon/PyKoSpacing.git

pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
# 필요한 객체 import
import pandas as pd
from konlpy.tag import Okt
from preprocessing import preprocess_data, tokenizing_data, get_label
from utils import seed_fix
import pandas as pd
import numpy as np
import random

import re
# 초기화 및 모델 학습
from gensim.models import word2vec
# 띄어쓰기
from pykospacing import Spacing
# 마춤뻡 검사기
from hanspell import spell_checker

from soynlp.normalizer import *

In [43]:
df1 = pd.read_csv("../data/" + "KorSTS/sts-train.tsv", sep='\t')
df2 = pd.read_csv("../data/" + "KorSTS/sts-dev.tsv", sep='\t')
df3 = pd.read_csv("../data/" + "KorSTS/sts-test.tsv", sep='\t')
df4 = pd.read_csv("../data/" + "whole_answers.csv", sep=',', encoding = 'utf-8')
data = open("../data/para_kqc_sim_data.txt", 'r', encoding='utf-8')
lines = data.readlines()
random.shuffle(lines)
train_data, test_data = preprocess_data(lines)

seed_fix(42)


train_contexts1 = list(
    df1.iloc[i].sentence1 for i in range(len(df1))
    )  # set 은 매번 순서가 바뀌므로
train_contexts2 = list(df1.iloc[j].sentence2 for j in range(len(df1)))
dev_contexts1 = list(
    df2.iloc[i].sentence1 for i in range(len(df2))
        )  # set 은 매번 순서가 바뀌므로
dev_contexts2 = list(df2.iloc[j].sentence2 for j in range(len(df2)))
test_contexts1 = list(
    df3.iloc[i].sentence1 for i in range(len(df3))
    )
test_contexts2 = list(df3.iloc[j].sentence2 for j in range(len(df3)))

parpara_contexts1 = list(
    train_data.iloc[i].sent_a for i in range(len(train_data))
    )
parpara_contexts2 = list(train_data.iloc[j].sent_b for j in range(len(train_data)))

parpara_contexts3 = list(
    test_data.iloc[i].sent_a for i in range(len(test_data))
    )
parpara_contexts4 = list(test_data.iloc[j].sent_b for j in range(len(test_data)))
wai_contexts = list(
    df4["answeres"].iloc[i] for i in range(len(df4))
    )


contexts = train_contexts1 + train_contexts2 + dev_contexts1 + dev_contexts2 + test_contexts1 + test_contexts2 + parpara_contexts1 + parpara_contexts2 + parpara_contexts3 + parpara_contexts4 + wai_contexts

18964it [00:00, 115994.31it/s]


중복 제거 후 학습 데이터셋 : 15171
중복 제거 후 테스트 데이터셋 : 3792
null 제거 후 학습 데이터셋 : 15171
null 제거 후 테스트 데이터셋 : 3792


In [12]:
contexts

['비행기가 이륙하고 있다.',
 '한 남자가 큰 플루트를 연주하고 있다.',
 '한 남자가 피자에 치즈를 뿌려놓고 있다.',
 '세 남자가 체스를 하고 있다.',
 '한 남자가 첼로를 연주하고 있다.',
 '몇몇 남자들이 싸우고 있다.',
 '남자가 담배를 피우고 있다.',
 '남자가 피아노를 치고 있다.',
 '한 남자가 기타를 치고 노래를 부르고 있다.',
 '사람이 고양이를 천장에 던지고 있다.',
 '그 남자는 다른 남자를 막대기로 때렸다.',
 '한 여성이 아기를 안아서 캥거루를 안는다.',
 '남자가 플루트를 연주하고 있다.',
 '사람이 종이 한 장을 접고 있다.',
 '한 남자가 도로를 달리고 있다.',
 '개가 베이컨을 등에서 떼려고 하고 있다.',
 '북극곰이 눈 위에서 미끄러지고 있다.',
 '여자가 글을 쓰고 있다.',
 '고양이가 아기의 얼굴을 문지르고 있다.',
 '남자가 말을 타고 있다.',
 '한 남자가 냄비에 기름을 부어 넣는다.',
 '남자가 기타를 치고 있다.',
 '판다가 슬라이드 아래로 미끄러져 내려간다.',
 '여자가 뭔가를 먹고 있다.',
 '한 여자가 감자 껍질을 벗긴다.',
 '그 소년은 자전거에서 떨어졌다.',
 '여자가 플루트를 연주하고 있다.',
 '토끼가 독수리에게서 뛰고 있다.',
 '여자가 빵을 곁들인 돼지고기를 튀기고 있다.',
 '한 소녀가 연을 날리고 있다.',
 '한 남자가 기계식 황소를 타고 있다.',
 '남자가 기타를 연주하고 있다.',
 '한 여성이 다른 여성들과 춤추고 노래하고 있다.',
 '남자가 빵을 자르고 있다.',
 '남자가 팬에 기름을 부어 넣고 있다.',
 '사자가 사람들과 놀고 있다.',
 '개가 스케이트보드를 탄다.',
 '누군가가 동상을 조각하고 있다.',
 '한 여성이 양파를 자르고 있다.',
 '여자는 새우 껍질을 벗긴다.',
 '한 여성이 생선 튀김을 하고 있다.',
 '한 여성이 일렉트릭 기타를 연주하고 있다.',
 '새끼 호랑이가 공을 가지고 놀고 있다.',
 '사

In [44]:
for word in contexts:
    if type(word) == float:
        contexts.remove(word)

In [45]:
def remove_repeat_char(contexts):
    preprocessed_text = []
    for text in contexts:
        text = repeat_normalize(text, num_repeats=2).strip()
        if text:
            preprocessed_text.append(text)

    return preprocessed_text

In [46]:
remove_repeat_char(contexts)

['비행기가 이륙하고 있다.',
 '한 남자가 큰 플루트를 연주하고 있다.',
 '한 남자가 피자에 치즈를 뿌려놓고 있다.',
 '세 남자가 체스를 하고 있다.',
 '한 남자가 첼로를 연주하고 있다.',
 '몇몇 남자들이 싸우고 있다.',
 '남자가 담배를 피우고 있다.',
 '남자가 피아노를 치고 있다.',
 '한 남자가 기타를 치고 노래를 부르고 있다.',
 '사람이 고양이를 천장에 던지고 있다.',
 '그 남자는 다른 남자를 막대기로 때렸다.',
 '한 여성이 아기를 안아서 캥거루를 안는다.',
 '남자가 플루트를 연주하고 있다.',
 '사람이 종이 한 장을 접고 있다.',
 '한 남자가 도로를 달리고 있다.',
 '개가 베이컨을 등에서 떼려고 하고 있다.',
 '북극곰이 눈 위에서 미끄러지고 있다.',
 '여자가 글을 쓰고 있다.',
 '고양이가 아기의 얼굴을 문지르고 있다.',
 '남자가 말을 타고 있다.',
 '한 남자가 냄비에 기름을 부어 넣는다.',
 '남자가 기타를 치고 있다.',
 '판다가 슬라이드 아래로 미끄러져 내려간다.',
 '여자가 뭔가를 먹고 있다.',
 '한 여자가 감자 껍질을 벗긴다.',
 '그 소년은 자전거에서 떨어졌다.',
 '여자가 플루트를 연주하고 있다.',
 '토끼가 독수리에게서 뛰고 있다.',
 '여자가 빵을 곁들인 돼지고기를 튀기고 있다.',
 '한 소녀가 연을 날리고 있다.',
 '한 남자가 기계식 황소를 타고 있다.',
 '남자가 기타를 연주하고 있다.',
 '한 여성이 다른 여성들과 춤추고 노래하고 있다.',
 '남자가 빵을 자르고 있다.',
 '남자가 팬에 기름을 부어 넣고 있다.',
 '사자가 사람들과 놀고 있다.',
 '개가 스케이트보드를 탄다.',
 '누군가가 동상을 조각하고 있다.',
 '한 여성이 양파를 자르고 있다.',
 '여자는 새우 껍질을 벗긴다.',
 '한 여성이 생선 튀김을 하고 있다.',
 '한 여성이 일렉트릭 기타를 연주하고 있다.',
 '새끼 호랑이가 공을 가지고 놀고 있다.',
 '사

In [50]:
# 한국어 정규식 확인 + ㅏ, ㅋ 이런 거 제거
def regular_check(contexts):
    preprocessed_text = []
    for text in contexts:
        text = re.sub('[ㄱ-ㅎㅏ-ㅣ]', '', text)
        text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', text)
        if text:
            preprocessed_text.append(text)

    return preprocessed_text

In [51]:
contexts = regular_check(contexts)

In [53]:
def clean_punc(texts):
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

    preprocessed_text = []
    for text in texts:
        for p in punct_mapping:
            text = text.replace(p, punct_mapping[p])
        text = text.strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [54]:
contexts = clean_punc(contexts)

In [57]:
def remove_repeated_spacing(texts):
    """
    두 개 이상의 연속된 공백을 하나로 치환합니다.
    ``오늘은    날씨가   좋다.`` -> ``오늘은 날씨가 좋다.``
    """
    preprocessed_text = []
    for text in texts:
        text = re.sub(r"\s+", " ", text).strip()
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [58]:
contexts = remove_repeated_spacing(contexts)

In [64]:
def spacing_sent(texts):
    """
    띄어쓰기를 보정합니다.
    """
    preprocessed_text = []
    spacing = Spacing()
    for text in texts:
        text = spacing(text)
        if text:
            preprocessed_text.append(text)
    return preprocessed_text

In [65]:
contexts = spacing_sent(contexts)

In [ ]:
def spell_check_sent(texts):
    """
    맞춤법을 보정합니다.
    """
    preprocessed_text = []
    for text in texts:
        try:
            spelled_sent = spell_checker.check(text)
            checked_sent = spelled_sent.checked 
            if checked_sent:
                preprocessed_text.append(checked_sent)
        except:
            preprocessed_text.append(text)
    return preprocessed_text

In [ ]:
contexts = spell_check_sent(contexts)

In [ ]:
import pickle

with open("contexts.pkl", "wb") as f:
    pickle.dump(contexts, f)

In [59]:
contexts[-10:]

['무역과 교류가 활발했죠',
 '아라비아 상인들과 교역을 했습니다',
 '고려 시대에는 한반도 내에서의 교류 말고도 멀리에서도 교류를 했다는 것을 알 수 있다 배를 타고 멀리 갈 수 있는 장비가 갖춰져 있었다는 것이다',
 '서양의 문물을 잘 받아들였다',
 '고려시대에 해양이나 육로를 통해 저 먼 아랍 지역까지 교류릉 했다는 사실을 알 수 있다',
 '고려의 물건들이 해외 각지로 팔려나갓다',
 '무역이 활발했다',
 '기억이 안나요죄송합니다',
 '대외 무역 활발',
 '다른 나라와의 무역이 활발하게 이루어졌다']

In [4]:
pos_tagger = Okt()

def tokenize(doc):
    # norm, stem은 optional
    if type(doc) is not str:
        return []
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

tokenized_list = []

for i in range(len(contexts)):    
    review = tokenize(contexts[i])
    tokenized_list.append(review) # 토큰화 된 리뷰를 리스트에 담아줌
    
print(tokenized_list[:10])

[['비행기/Noun', '가/Josa', '이륙/Noun', '하고/Josa', '있다/Adjective', './Punctuation'], ['하다/Verb', '남자/Noun', '가/Josa', '크다/Verb', '플루트/Noun', '를/Josa', '연주/Noun', '하고/Josa', '있다/Adjective', './Punctuation'], ['하다/Verb', '남자/Noun', '가/Josa', '피자/Noun', '에/Josa', '치즈/Noun', '를/Josa', '뿌리다/Verb', '있다/Adjective', './Punctuation'], ['세/Noun', '남자/Noun', '가/Josa', '체스/Noun', '를/Josa', '하다/Verb', '있다/Adjective', './Punctuation'], ['하다/Verb', '남자/Noun', '가/Josa', '첼로/Noun', '를/Josa', '연주/Noun', '하고/Josa', '있다/Adjective', './Punctuation'], ['몇몇/Noun', '남자/Noun', '들/Suffix', '이/Josa', '싸우다/Verb', '있다/Adjective', './Punctuation'], ['남자/Noun', '가/Josa', '담배/Noun', '를/Josa', '피우다/Verb', '있다/Adjective', './Punctuation'], ['남자/Noun', '가/Josa', '피아노/Noun', '를/Josa', '치고/Josa', '있다/Adjective', './Punctuation'], ['하다/Verb', '남자/Noun', '가/Josa', '기타/Noun', '를/Josa', '치고/Josa', '노래/Noun', '를/Josa', '부르다/Verb', '있다/Adjective', './Punctuation'], ['사람/Noun', '이/Josa', '고양이/Noun', '를/Josa', '천장/Noun', '에/Josa', '던지

In [5]:
#하이퍼 파라메터 값 지정
num_features = 200 # 문자 벡터 차원수
min_word_count = 40 # 최소 문자 수
num_workers = 4 # 병렬 쓰레드 수
context = 10 # window size
downsampling = 1e-3 # 문자 빈도 수 downsample

#모델 학습
model = word2vec.Word2Vec(tokenized_list,
                            workers = num_workers,
                            vector_size = num_features,
                            min_count = min_word_count,
                            window = context,
                            sample = downsampling
                        )



In [6]:
# 학습이 완료 되면 필요없는 메모리를 unload 시킨다.
model.init_sims(replace=True)

model_name = 'Word2Vec_Kor'
# model_name = '300features_50minwords_20text'
model.save(model_name)
model.wv.save_word2vec_format('my.embedding', binary=False)

/tmp/ipykernel_26413/3082260096.py:2: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [9]:
def cosine_similarity(a, b):

    #return np.inner(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

father_vec = model.wv.get_vector('아버지/Noun')
mather_vec = model.wv.get_vector('어머니/Noun')
father2_vec = model.wv.get_vector('아버님/Noun')
print(cosine_similarity(father_vec, mather_vec))
print(cosine_similarity(father_vec, father2_vec))

0.8501051
0.81097925


In [52]:
import re

class Keyword_similarity:
    
    def __init__(self, model, threshold = 0.5, pos_tagger=None, lemmatizer=None):
        self.model = model
        self.threshold = threshold
        self.pos_tagger = pos_tagger
        self.lemmatizer = lemmatizer
    
    ### 키워드랑 문장이 리스트로 들어온다.
    def tokenize(self, doc):
    # norm, stem은 optional
        if type(doc) is not str:
            return []
        return ['/'.join(t) for t in self.pos_tagger.pos(doc, norm=True, stem=True)]

    def keyword_score(self, keyword_list, sentence_list):
        idx_list = []
        for keyword in keyword_list:
            idx_list.append(self.get_keyword_score_list(keyword, sentence_list))

        return idx_list

    ## 키워드 하나에 대해서 답안들에 대해 조사한다.
    def get_keyword_score_list(self, keyword, sentence_list):
        idx_list = []
        for sentence in sentence_list:
            idx_list.append(self.keyword_one_sentence(keyword, sentence))
        
        return idx_list

    ## 하나의 키워드에 대해서 그 문장의 것들에 대한 점수를 리스트로 반환한다.
    def keyword_one_sentence(self, keyword, sentence):
        pos_keyword = self.tokenize(keyword)
        keyword_vec = self.model.wv.get_vector(pos_keyword[0])
        
        cosine_list = []
        word_list = []
        for word in sentence:
            
            pos_word = self.tokenize(word)
            try: 
                word_vec = self.model.wv.get_vector(pos_word[0])
            except:
                continue
            if cosine_similarity(keyword_vec, word_vec) > self.threshold and pos_word not in word_list:
                results = re.finditer(word, sentence)
                for matched_key in results:
                    cosine_list.append([matched_key.start(), matched_key.end()])
                
                word_list.append(pos_word)
            
        return cosine_list


In [1]:
sentence_list = ["어느 곳에 빵이 더 맛있는 지 알 수 있다."]
keyword_list = ["가격"]

KS = Keyword_similarity(model, 0.5, pos_tagger)
KS.keyword_one_sentence("가격", "어느 곳에 빵이 더 맛있는지 알 수 있다.")

NameError: name 'Keyword_similarity' is not defined